In [1]:
import findspark
findspark.init()
from pyspark import SparkContext, SparkFiles
from pyspark.sql import SQLContext, SparkSession

주의! SparkContext()는 최초 실행 후 다시 실행하면 에러가 발생한다.
> 🤔 왜 ?

In [2]:
sc = SparkContext() 
sqlContext = SQLContext(sc)

In [3]:
url = "https://raw.githubusercontent.com/guru99-edu/R-Programming/master/adult_data.csv"
sc.addFile(url)
sqlContext = SQLContext(sc)

In [4]:
df = sqlContext.read.csv(
    SparkFiles.get("adult_data.csv"), 
    header=True, 
    inferSchema=True
)

### select

In [5]:
df.select('age', 'race', 'income').show()

+---+------------------+------+
|age|              race|income|
+---+------------------+------+
| 25|             Black| <=50K|
| 38|             White| <=50K|
| 28|             White|  >50K|
| 44|             Black|  >50K|
| 18|             White| <=50K|
| 34|             White| <=50K|
| 29|             Black| <=50K|
| 63|             White|  >50K|
| 24|             White| <=50K|
| 55|             White| <=50K|
| 65|             White|  >50K|
| 36|             White| <=50K|
| 26|             White| <=50K|
| 58|             White| <=50K|
| 48|             White|  >50K|
| 43|             White|  >50K|
| 20|             White| <=50K|
| 43|             White| <=50K|
| 37|             White| <=50K|
| 40|Asian-Pac-Islander|  >50K|
+---+------------------+------+
only showing top 20 rows



### where 
```filter()```

In [6]:
df.filter('age > 70').collect()[0]

Row(x=23, age=72, workclass='?', fnlwgt=132015, education='7th-8th', educational-num=4, marital-status='Divorced', occupation='?', relationship='Not-in-family', race='White', gender='Female', capital-gain=0, capital-loss=0, hours-per-week=6, native-country='United-States', income='<=50K')

and 조건 / or 조건

In [7]:
df.filter('age > 70 and age < 80').collect()[0]

Row(x=23, age=72, workclass='?', fnlwgt=132015, education='7th-8th', educational-num=4, marital-status='Divorced', occupation='?', relationship='Not-in-family', race='White', gender='Female', capital-gain=0, capital-loss=0, hours-per-week=6, native-country='United-States', income='<=50K')

In [8]:
df.filter('age < 20 or age > 80').collect()[1]

Row(x=39, age=17, workclass='Private', fnlwgt=269430, education='10th', educational-num=6, marital-status='Never-married', occupation='Machine-op-inspct', relationship='Not-in-family', race='White', gender='Male', capital-gain=0, capital-loss=0, hours-per-week=40, native-country='United-States', income='<=50K')

### GROUP BY

```groupBY()```

In [9]:
df.groupBy('gender').avg('age').collect()

[Row(gender='Female', avg(age)=36.92798913043478),
 Row(gender='Male', avg(age)=39.49439509954058)]

groupBy 기준 여러개를 설정할 수 있다.

In [10]:
df.groupBy('gender', 'race').avg('age').collect()

[Row(gender='Male', race='White', avg(age)=39.70450669914738),
 Row(gender='Female', race='Asian-Pac-Islander', avg(age)=35.657640232108314),
 Row(gender='Female', race='White', avg(age)=36.88293544177478),
 Row(gender='Female', race='Amer-Indian-Eskimo', avg(age)=36.23783783783784),
 Row(gender='Male', race='Other', avg(age)=35.167330677290835),
 Row(gender='Male', race='Black', avg(age)=37.922591501893145),
 Row(gender='Male', race='Asian-Pac-Islander', avg(age)=38.9940119760479),
 Row(gender='Male', race='Amer-Indian-Eskimo', avg(age)=36.98947368421052),
 Row(gender='Female', race='Other', avg(age)=31.212903225806453),
 Row(gender='Female', race='Black', avg(age)=37.90597920277296)]

물론 구하려는 변수에도 여러개 입력 가능

In [11]:
df.groupBy('gender').avg('age', 'fnlwgt').show()

+------+-----------------+------------------+
|gender|         avg(age)|       avg(fnlwgt)|
+------+-----------------+------------------+
|Female|36.92798913043478|185504.47171442688|
|  Male|39.49439509954058| 191727.0216232772|
+------+-----------------+------------------+



### order
ORDER BY는 .sort()로 할 수 있다. 

desc()로 감싸면 descending 가능

In [13]:
from pyspark.sql.functions import desc

df.groupBy('gender', 'race').avg('age').sort(desc('gender'), 'race').show()

+------+------------------+------------------+
|gender|              race|          avg(age)|
+------+------------------+------------------+
|  Male|Amer-Indian-Eskimo| 36.98947368421052|
|  Male|Asian-Pac-Islander|  38.9940119760479|
|  Male|             Black|37.922591501893145|
|  Male|             Other|35.167330677290835|
|  Male|             White| 39.70450669914738|
|Female|Amer-Indian-Eskimo| 36.23783783783784|
|Female|Asian-Pac-Islander|35.657640232108314|
|Female|             Black| 37.90597920277296|
|Female|             Other|31.212903225806453|
|Female|             White| 36.88293544177478|
+------+------------------+------------------+



### join을 알아보자
default inner. 

Must be one of: inner, cross, outer, full, full_outer, left, left_outer, right, right_outer, left_semi, and left_anti.

In [14]:
ss = SparkSession.builder.getOrCreate()

In [15]:
df_1 = ss.createDataFrame(
    data = [
        ('james', 35),  
        ('leonard', 28),
        ('paul', 34),
        ('westbrook', 31),
        ('siakam', 28),
        ('jokic', 25)
    ]
    , schema = ["NAME", "AGE"]
)

In [16]:
df_1.show()

+---------+---+
|     NAME|AGE|
+---------+---+
|    james| 35|
|  leonard| 28|
|     paul| 34|
|westbrook| 31|
|   siakam| 28|
|    jokic| 25|
+---------+---+



In [17]:
df_2 = ss.createDataFrame(
    data = [
        ('james', 'lal'),  
        ('paul', 'okc'),
        ('westbrook', 'hou'),
        ('siakam', 'tor'),
        ('jokic', 'den'),
        ('whiteside', 'cle')
    ]
    , schema = ["NAME", "TEAM"]
)

In [18]:
df_2.show()

+---------+----+
|     NAME|TEAM|
+---------+----+
|    james| lal|
|     paul| okc|
|westbrook| hou|
|   siakam| tor|
|    jokic| den|
|whiteside| cle|
+---------+----+



- Inner Join

In [19]:
inner_join = df_1.join(df_2, df_1.NAME == df_2.NAME, 'inner').select(df_1.NAME, df_1.AGE, df_2.TEAM)
inner_join.show()

+---------+---+----+
|     NAME|AGE|TEAM|
+---------+---+----+
|    jokic| 25| den|
|     paul| 34| okc|
|westbrook| 31| hou|
|   siakam| 28| tor|
|    james| 35| lal|
+---------+---+----+



- Left Join

In [21]:
left_join = df_1.join(df_2, df_1.NAME == df_2.NAME, 'left').select(df_1.NAME, df_1.AGE, df_2.TEAM)
left_join.show()

+---------+---+----+
|     NAME|AGE|TEAM|
+---------+---+----+
|    jokic| 25| den|
|  leonard| 28|null|
|     paul| 34| okc|
|westbrook| 31| hou|
|   siakam| 28| tor|
|    james| 35| lal|
+---------+---+----+



- Rigth Join

In [22]:
right_join = df_1.join(df_2, df_1.NAME == df_2.NAME, 'right').select(df_1.NAME, df_1.AGE, df_2.TEAM)
right_join.show()

+---------+----+----+
|     NAME| AGE|TEAM|
+---------+----+----+
|    jokic|  25| den|
|     paul|  34| okc|
|westbrook|  31| hou|
|   siakam|  28| tor|
|    james|  35| lal|
|     null|null| cle|
+---------+----+----+



- Outer Join

In [26]:
outer_join = df_1.join(df_2, df_1.NAME == df_2.NAME, 'outer').select(df_1.NAME, df_1.AGE, df_2.TEAM)
outer_join.show()

+---------+----+----+
|     NAME| AGE|TEAM|
+---------+----+----+
|    jokic|  25| den|
|  leonard|  28|null|
|     paul|  34| okc|
|westbrook|  31| hou|
|   siakam|  28| tor|
|    james|  35| lal|
|     null|null| cle|
+---------+----+----+



- Cross Join

In [33]:
df_1.crossJoin(df_2.select("TEAM")).select("NAME", "AGE", "TEAM").show()

+---------+---+----+
|     NAME|AGE|TEAM|
+---------+---+----+
|    james| 35| lal|
|    james| 35| okc|
|    james| 35| hou|
|    james| 35| tor|
|    james| 35| den|
|    james| 35| cle|
|  leonard| 28| lal|
|  leonard| 28| okc|
|  leonard| 28| hou|
|  leonard| 28| tor|
|  leonard| 28| den|
|  leonard| 28| cle|
|     paul| 34| lal|
|     paul| 34| okc|
|     paul| 34| hou|
|     paul| 34| tor|
|     paul| 34| den|
|     paul| 34| cle|
|westbrook| 31| lal|
|westbrook| 31| okc|
+---------+---+----+
only showing top 20 rows

